In [1]:
import timm
import torch
import torch.nn as nn
import numpy as np

In [2]:
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

model = timm.create_model('vit_base_patch16_224', pretrained=True)

config = resolve_data_config({}, model=model)
transform = create_transform(**config)

In [16]:
dict(dict(dict(model.named_children())['blocks'].named_children())['0'].named_children())

{'norm1': LayerNorm((768,), eps=1e-06, elementwise_affine=True),
 'attn': Attention(
   (qkv): Linear(in_features=768, out_features=2304, bias=True)
   (q_norm): Identity()
   (k_norm): Identity()
   (attn_drop): Dropout(p=0.0, inplace=False)
   (proj): Linear(in_features=768, out_features=768, bias=True)
   (proj_drop): Dropout(p=0.0, inplace=False)
 ),
 'ls1': Identity(),
 'drop_path1': Identity(),
 'norm2': LayerNorm((768,), eps=1e-06, elementwise_affine=True),
 'mlp': Mlp(
   (fc1): Linear(in_features=768, out_features=3072, bias=True)
   (act): GELU(approximate='none')
   (drop1): Dropout(p=0.0, inplace=False)
   (norm): Identity()
   (fc2): Linear(in_features=3072, out_features=768, bias=True)
   (drop2): Dropout(p=0.0, inplace=False)
 ),
 'ls2': Identity(),
 'drop_path2': Identity()}

In [5]:
layer = 0
block = dict(dict(dict(model.named_children())['blocks'].named_children())[str(layer)].named_children())
# list(block['attn'].children())[0]
ls = [list(block['attn'].children())[0], list(block['attn'].children())[4],list(block['mlp'].children())[0], list(block['mlp'].children())[4]]
ls[0].weight.clone().detach().numpy().T

Linear(in_features=768, out_features=2304, bias=True)